<img src='10215_1.jpg'>

<h1>Формирование датасета с появившейся дневной задолженностью за аренды автомобилей</h1>

In [1]:
import mysql.connector

In [2]:
mydb = mysql.connector.connect(
    host='localhost',
    user='root',
    passwd='yk5Qa9Xq1w5tBc9Ye',
    database='mega'
)

In [3]:
import pandas as pd
import math
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import datetime
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
import plotly.graph_objs as go

In [4]:
revenue_per_month_and_citys = pd.read_sql('SELECT city, created_at_format, SUM(total_debt) AS total_debt_total  \
                                        FROM \
                                        (SELECT DISTINCT reg_t.title_ru AS city, u_payments.operation_id, users_transactions.paid_by_bonus, \
                                        MAX(u_payments.amount), MAX(u_payments.amount)-users_transactions.paid_by_bonus AS total_debt, \
                                        MIN(FROM_UNIXTIME(u_payments.created_at, "%Y-%m-%d")) AS created_at_format \
                                        FROM mega.user_payments AS u_payments \
                                        INNER JOIN mega.regions AS reg_t ON u_payments.region_id = reg_t.id \
                                        INNER JOIN mega.user_operations AS users_transactions ON u_payments.operation_id = users_transactions.id \
                                        WHERE users_transactions.reference_id IS NOT NULL AND users_transactions.operation_type IN ("minutes", "hold", "after-packet") AND\
                                        u_payments.merchant_type IS NULL \
                                        AND u_payments.status = 1 AND \
                                        u_payments.operation_id NOT IN (SELECT debt_start_date.operation_id \
                                                        FROM \
                                                        (SELECT DISTINCT user_id, operation_id, MAX(FROM_UNIXTIME(created_at, "%Y-%m-%d")) \
                                                        AS created_at_format \
                                                        FROM mega.user_payments \
                                                        GROUP BY user_id, operation_id, status, amount_return, merchant_type \
                                                        HAVING (status IN (2,5,13)) AND (amount_return IS NULL) AND (merchant_type IS NULL)) \
                                                        AS debt_final_date \
                                                        INNER JOIN \
                                                        (SELECT DISTINCT user_id, operation_id, MIN(FROM_UNIXTIME(created_at, "%Y-%m-%d")) \
                                                        AS created_at_format \
                                                        FROM mega.user_payments \
                                                        GROUP BY user_id, operation_id, status, merchant_type \
                                                        HAVING (status = 1) AND (merchant_type IS NULL)) AS debt_start_date ON \
                                                        debt_final_date.user_id = debt_start_date.user_id AND \
                                                        debt_final_date.operation_id = debt_start_date.operation_id AND \
                                                        debt_final_date.created_at_format = debt_start_date.created_at_format) \
                                        GROUP BY city, u_payments.operation_id) AS total_table \
                                        GROUP BY city, created_at_format \
                                        HAVING SUM(total_debt) > 0 \
                                        ORDER BY created_at_format, city',mydb)

In [5]:
revenue_per_month_and_citys.to_csv('total_debt_received_for_the_days_rent.csv')